## LangChain v0.1.0

#### LangChain's Blog Post and Video about the new release
* Released Jan 8, 2024.
* [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
* [YouTube Walkthroug](https://www.youtube.com/watch?v=Fk_zZr2DbQY&list=PLfaIDFEXuae0gBSJ9T0w7cu7iJZbH3T31)

#### Summary
* First stable version.
* Fully backwards compatible.
* Both in Python and Javascript.
* Improved functionality.
* Improved documentation.

#### Main change: the old LangChain package is splitted
* lanchain-core: core functionality.
* langchain-community: third-party integrations
* standalone partner packages
    * example: langchain-openai

#### In theory, all previous LangChain projects should work
* In practice, this does not seem credible.

#### Example using langchain-core

#### Example using langchain-community

#### Example using langchain-openai

## New Quickstart
* Setup LangChain, LangSmith and LangServe.
* Use basic componets:
    * Prompt templates.
    * Models.
    * Output parsers.
* Use LCEL.
* Build a simple app.
* Trace your app with LangSmith.
* Serve your app with LangServe. 

#### Create a new virtual environment

#### Create a .env file with the OpenAI credentials

#### LangChain Installation

In [1]:
#!pip install langchain

#### If you set the LangSmith credentials in the .env file, LangChain will start logging traces.

If you do want to use LangSmith, after you sign up at LangSmith, make sure to set your environment variables in your .env file to start logging traces:

LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=...

#### What we will cover
* Simple LLM chain.
* Retrieval chain.
* Conversation Retrieval chain.
* Agent.

#### Use the new langchain_openai

In [7]:
#!pip install langchain-openai

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI()

In [4]:
llm.invoke("What was the name of Napoleon's wife?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

#### Use the new lanchain_core

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [9]:
my_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are friendly assisstant."),
    ("user", "{input}")
])

#### Create a chain with LCEL

In [10]:
my_chain = my_prompt_template | llm 

In [11]:
my_chain.invoke({"input": "Where was Napoleon defeated?"})

AIMessage(content='Napoleon was famously defeated at the Battle of Waterloo, which took place near the town of Waterloo in present-day Belgium on June 18, 1815.')

#### Create an Output Parser to convert the chat message to a string

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
to_string_output_parser = StrOutputParser()

#### Add the Output Parser to the Chain

In [14]:
my_chain = my_prompt_template | llm | to_string_output_parser

In [15]:
my_chain.invoke({"input": "Where was the main victory of Napoleon?"})

"One of Napoleon's most significant victories was the Battle of Austerlitz, also known as the Battle of the Three Emperors, which took place on December 2, 1805. The battle was fought near the town of Austerlitz in the present-day Czech Republic. Napoleon's French forces defeated a combined army of Russian and Austrian troops, securing a decisive victory for the French Empire."

#### Simple RAG: Private Document, Splitter, Vector Database and Retrieval Chain.

We can load our private document from different sources (from a file, from the web, etc). In this example we will load our private data from the web using WebBaseLoader. In order to use WebBaseLoader we will need to install BeautifulSoup:

In [17]:
#!pip install beautifulsoup4

To import WebBaseLoader, we will **use the new langchain_community**:

In [18]:
from langchain_community.document_loaders import WebBaseLoader

In [19]:
my_loader = WebBaseLoader("https://aiaccelera.com/ai-consulting-for-businesses/")

In [20]:
my_private_docs = my_loader.load()

We will use OpenAIEmbeddings to convert our private docs to numbers:

In [21]:
from langchain_openai import OpenAIEmbeddings

In [22]:
my_embeddings = OpenAIEmbeddings()

We will use FAISS as vector database:

In [24]:
#!pip install faiss-cpu

In [25]:
from langchain_community.vectorstores import FAISS

We will use RecursiveCharacterTextSplitter to divide the private docs into smaller text chunks:

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [28]:
my_text_splitter = RecursiveCharacterTextSplitter()

In [29]:
my_text_chunks = my_text_splitter.split_documents(my_private_docs)

In [30]:
my_vector_database = FAISS.from_documents(my_text_chunks, my_embeddings)

Now we will create a chain that takes the question and the retrieved documents and generates an answer:

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [32]:
my_prompt_template = ChatPromptTemplate.from_template(
    """Answer the following question based only on the 
    provided context:

    <context>
    {context}
    </context>

    Question: {input}"""
)

In [36]:
my_document_answering_chain = create_stuff_documents_chain(llm, my_prompt_template)

Next we will create the retrieval chain:

In [37]:
from langchain.chains import create_retrieval_chain

In [38]:
my_retriever = my_vector_database.as_retriever()

In [39]:
my_retrieval_chain = create_retrieval_chain(my_retriever, my_document_answering_chain)

We can now start using the retrieval chain:

In [45]:
response = my_retrieval_chain.invoke({
    "input": "Summarize the provided context in less than 100 words"
})

In [46]:
print(response["answer"])

The provided context describes a company that offers AI consulting and development services. They help businesses identify use cases for AI and develop LLM (Generative AI) applications. The company follows a comprehensive approach, starting from idea validation to deployment and ongoing support. They emphasize the benefits of using generative AI, such as increased creativity, efficiency, enhanced decision making, and personalized experiences. The company also offers maintenance and support for AI products. Overall, their goal is to help businesses harness the power of AI and unlock its potential for growth and innovation.
